In [1]:
# Install Ultralytics
!pip install -q ultralytics

# dataset https://www.kaggle.com/datasets/andrewmvd/helmet-detection

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.6 MB/s eta 0:00:00a 0:00:01


In [2]:
# Import necessary libraries
import os
import shutil
import yaml
import glob
from xml.dom import minidom
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# ==========================
# 📌 Step 1: YAML File Setup
# ==========================
yaml_path = "/kaggle/working/dataset.yaml"

data = {
    "train": "/kaggle/working/images",
    "val": "/kaggle/working/images",
    "test": "/kaggle/working/images",
    "nc": 2,
    "names": ["Without Helmet", "With Helmet"]
}

# Write dataset.yaml
with open(yaml_path, "w") as f:
    yaml.dump(data, f, default_flow_style=False)

print("✅ dataset.yaml created successfully!")

✅ dataset.yaml created successfully!


In [4]:
# ==========================
# 📌 Step 2: Prepare Directory Structure
# ==========================
label_dir = "/kaggle/working/labels"

# Remove old labels folder and create a new one
if os.path.exists(label_dir):
    shutil.rmtree(label_dir)
os.makedirs(label_dir, exist_ok=True)
print(f"✅ Created clean label directory: {label_dir}")

# Copy images from dataset
!cp -r /kaggle/input/helmet-detection/images/ /kaggle/working/

✅ Created clean label directory: /kaggle/working/labels


In [5]:
# ==========================
# 📌 Step 3: Convert XML Annotations to YOLO Format
# ==========================
lut = {"Without Helmet": 0, "With Helmet": 1}  # Class lookup table

def get_text(tag, parent):
    """Helper function to extract text from an XML tag."""
    return parent.getElementsByTagName(tag)[0].firstChild.data

def convert_coordinates(size, box):
    """Convert Pascal VOC bounding box format to YOLO format."""
    dw, dh = 1.0 / size[0], 1.0 / size[1]
    x = (box[0] + box[1]) / 2.0 * dw
    y = (box[2] + box[3]) / 2.0 * dh
    w = (box[1] - box[0]) * dw
    h = (box[3] - box[2]) * dh
    return x, y, w, h

def convert_xml2yolo(lut, input_folder, output_folder):
    """Converts all XML annotations in input_folder to YOLO format."""
    for fname in glob.glob(os.path.join(input_folder, "*.xml")):
        xmldoc = minidom.parse(fname)
        size = xmldoc.getElementsByTagName("size")[0]
        
        width, height = int(get_text("width", size)), int(get_text("height", size))
        fname_out = os.path.join(output_folder, os.path.basename(fname).replace(".xml", ".txt"))

        with open(fname_out, "w") as f:
            for item in xmldoc.getElementsByTagName("object"):
                class_name = get_text("name", item)
                if class_name not in lut:
                    print(f"⚠️ Warning: Unknown label '{class_name}' in {fname}. Skipping.")
                    continue

                # Bounding box extraction
                bbox = item.getElementsByTagName("bndbox")[0]
                xmin, xmax = float(get_text("xmin", bbox)), float(get_text("xmax", bbox))
                ymin, ymax = float(get_text("ymin", bbox)), float(get_text("ymax", bbox))
                
                bb = convert_coordinates((width, height), (xmin, xmax, ymin, ymax))
                f.write(f"{lut[class_name]} " + " ".join(f"{x:.6f}" for x in bb) + "\n")

        print(f"✅ Converted: {fname_out}")

# Convert XML files to YOLO format
convert_xml2yolo(lut, "/kaggle/input/helmet-detection/annotations", "/kaggle/working/labels")

✅ Converted: /kaggle/working/labels/BikesHelmets338.txt
✅ Converted: /kaggle/working/labels/BikesHelmets680.txt
✅ Converted: /kaggle/working/labels/BikesHelmets62.txt
✅ Converted: /kaggle/working/labels/BikesHelmets172.txt
✅ Converted: /kaggle/working/labels/BikesHelmets622.txt
✅ Converted: /kaggle/working/labels/BikesHelmets477.txt
✅ Converted: /kaggle/working/labels/BikesHelmets488.txt
✅ Converted: /kaggle/working/labels/BikesHelmets696.txt
✅ Converted: /kaggle/working/labels/BikesHelmets71.txt
✅ Converted: /kaggle/working/labels/BikesHelmets269.txt
✅ Converted: /kaggle/working/labels/BikesHelmets206.txt
✅ Converted: /kaggle/working/labels/BikesHelmets705.txt
✅ Converted: /kaggle/working/labels/BikesHelmets515.txt
✅ Converted: /kaggle/working/labels/BikesHelmets113.txt
✅ Converted: /kaggle/working/labels/BikesHelmets546.txt
✅ Converted: /kaggle/working/labels/BikesHelmets632.txt
✅ Converted: /kaggle/working/labels/BikesHelmets267.txt
✅ Converted: /kaggle/working/labels/BikesHelmets37

In [ ]:
# ==========================
# 📌 Step 4: Load and Train YOLO Model
# ==========================
model = YOLO("yolov8n.pt")  # Load pretrained YOLOv8 model

# Train the model
results = model.train(data=yaml_path, epochs=30)
print("✅ Training complete!")

Ultralytics 8.3.189 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0

In [ ]:
# ==========================
# 📌 Step 5: Run Inference on Test Images
# ==========================
test_images = glob.glob("/kaggle/input/helmet-detection/images/*.png")

for img_path in test_images[:3]:  # Limit to 3 images for quick testing
    results = model(img_path)
    for result in results:
        result.show()  # Display image
        result.save(filename=f"result_{os.path.basename(img_path)}")
    print(f"✅ Processed {img_path}")

In [ ]:
# ==========================
# 📌 Step 6: Export Model to ONNX
# ==========================
if model.export(format="onnx"):
    print("✅ Model exported to ONNX format successfully!")